In [1]:
%matplotlib inline
import os
import glob
import json
from collections import Counter, defaultdict

import urllib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Get the data from the PPLM github repo (https://github.com/uber-research/PPLM)

files = ['clickbait.csv',
         'computers.csv',
         'legal.csv',
         'military.csv',
         'negative.csv',
         'politics.csv',
         'positive.csv',
         'religion.csv',
         'science.csv',
         'space.csv']

col_lists = []
dfs = {}
base_url = 'https://raw.githubusercontent.com/uber-research/PPLM/master/human_annotation/pplm_labeled_csvs/'
for infile in files:
    print(infile)
    # some files uses a header, others don't
    if infile == 'clickbait.csv' or infile == 'positive.csv' or infile == 'negative.csv':
        df = pd.read_csv(base_url + infile, header=1, index_col=None)
    else:
        df = pd.read_csv(base_url + infile, header=0, index_col=None)   
    col_lists.append(list(df.columns))
    dfs[infile] = df

for i in range(len(col_lists[0])):
    for cols in col_lists:
        print(cols[i], end=' ')
        print()    

clickbait.csv
computers.csv
legal.csv
military.csv
negative.csv
politics.csv
positive.csv
religion.csv
science.csv
space.csv
A 
From files: computers_pickles/computers_R.p, computers_pickles/computers_A.p, computers_pickles/computers_RB.p, computers_pickles/computers_A,B.p 
From files: legal_words_pickles/legal_words_R.p, legal_words_pickles/legal_words_A.p, legal_words_pickles/legal_words_RB.p, legal_words_pickles/legal_words_A,B.p 
From files: military_pickles/military_R.p, military_pickles/military_A.p, military_pickles/military_RB.p, military_pickles/military_A,B.p 
A 
From files: politics_pickles/politics_R.p, politics_pickles/politics_A.p, politics_pickles/politics_RB.p, politics_pickles/politics_A,B.p 
A 
From files: religion_pickles/religion_R.p, religion_pickles/religion_A.p, religion_pickles/religion_RB.p, religion_pickles/religion_A,B.p 
From files: science_pickles/science_R.p, science_pickles/science_A.p, science_pickles/science_RB.p, science_pickles/science_A,B.p 
From fil

In [3]:
df.head(10)

,"From files: space_pickles/space_R.p, space_pickles/space_A.p, space_pickles/space_RB.p, space_pickles/space_A,B.p",Unnamed: 1,Which passage is more topic relevant?,How fluent is the passage of A?,How fluent is the passage of B?,Ben,Which passage is more topic relevant?.1,How fluent is the passage of A?.1,How fluent is the passage of B?.1,North,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,<|endoftext|>It has been shown that a signific...,"<|endoftext|>It has been shown that, if we are...",B,2.0,2.0,NaN,B,1.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19_131_633
1,<|endoftext|>It has been shown that the amount...,<|endoftext|>It has been shown that in the hum...,neither,4.0,4.0,NaN,neither,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18_389_15
2,"<|endoftext|>To summarise, it was an interesti...","<|endoftext|>To summarise, this is a story abo...",neither,3.0,3.0,NaN,A,3.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6_271_178
3,"<|endoftext|>Prior to this, the company was kn...","<|endoftext|>Prior to this, the company was kn...",B,3.0,3.0,NaN,B,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45_308_871
4,"<|endoftext|>More importantly, this is not an ...","<|endoftext|>More importantly, it would be the...",both,2.0,2.0,NaN,A,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10_751_705
5,<|endoftext|>The key aspect discussed in this ...,<|endoftext|>The key aspect discussed in this ...,neither,2.0,3.0,NaN,neither,5.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26_120_81
6,<|endoftext|>Foundational to this is the fact ...,<|endoftext|>Foundational to this is a basic u...,both,4.0,3.0,NaN,both,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12_793_399
7,<|endoftext|>This essay discusses the history ...,<|endoftext|>This essay discusses the relation...,neither,4.0,4.0,NaN,A,3.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21_449_764
8,"<|endoftext|>In summary, it seems that the maj...",<|endoftext|>In summary:\n\n• It's possible th...,neither,2.0,3.0,NaN,neither,2.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43_377_58
9,"<|endoftext|>To review, this is a story about ...","<|endoftext|>To review, the two men were sitti...",neither,3.0,2.0,NaN,neither,4.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3_460_617


In [4]:
len(df)

365

In [5]:
df.columns

Index(['From files: space_pickles/space_R.p, space_pickles/space_A.p, space_pickles/space_RB.p, space_pickles/space_A,B.p',
       'Unnamed: 1', 'Which passage is more topic relevant?',
       'How fluent is the passage of A?', 'How fluent is the passage of B?',
       'Ben', 'Which passage is more topic relevant?.1',
       'How fluent is the passage of A?.1',
       'How fluent is the passage of B?.1', 'North',
       'Which passage is more topic relevant?.2',
       'How fluent is the passage of A?.2',
       'How fluent is the passage of B?.2', 'Hunter', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [6]:
df.iloc[0]

From files: space_pickles/space_R.p, space_pickles/space_A.p, space_pickles/space_RB.p, space_pickles/space_A,B.p    <|endoftext|>It has been shown that a signific...
Unnamed: 1                                                                                                           <|endoftext|>It has been shown that, if we are...
Which passage is more topic relevant?                                                                                                                                B
How fluent is the passage of A?                                                                                                                                    2.0
How fluent is the passage of B?                                                                                                                                    2.0
Ben                                                                                                                                                                Na

In [7]:
df.iloc[1]

From files: space_pickles/space_R.p, space_pickles/space_A.p, space_pickles/space_RB.p, space_pickles/space_A,B.p    <|endoftext|>It has been shown that the amount...
Unnamed: 1                                                                                                           <|endoftext|>It has been shown that in the hum...
Which passage is more topic relevant?                                                                                                                          neither
How fluent is the passage of A?                                                                                                                                    4.0
How fluent is the passage of B?                                                                                                                                    4.0
Ben                                                                                                                                                                Na

In [8]:
# Look at the number of times each annotator appears
name_counter = Counter()
for infile in files:
    df = dfs[infile]
    columns = list(df.columns)        
    annotators = [columns[i].title() for i in [5, 9, 13]]
    name_counter.update(annotators)
for n, c in name_counter.most_common():
    print(n, c)

Gabe 4
Ben 4
Prabha 3
North 3
Unnamed: 13 2
Donnie 2
Shannon 2
Spencer 2
Ethan 2
Stephen 2
Raymis 1
Ally 1
Zakk 1
Hunter 1


In [9]:
files

['clickbait.csv',
 'computers.csv',
 'legal.csv',
 'military.csv',
 'negative.csv',
 'politics.csv',
 'positive.csv',
 'religion.csv',
 'science.csv',
 'space.csv']

In [10]:
base = os.path.basename('clickbait.csv')
basename = os.path.splitext(base)[0]

In [11]:
dfs.keys()

dict_keys(['clickbait.csv', 'computers.csv', 'legal.csv', 'military.csv', 'negative.csv', 'politics.csv', 'positive.csv', 'religion.csv', 'science.csv', 'space.csv'])

In [12]:
# assign model IDs based on information in the dataset code/documentation
models = {0: 'B', 1: 'BC', 2: 'BR', 3: 'BCR'}
#pair_model = {0: 'B-BC', 1: 'B-BR', 2: 'B-BCR', 3: 'BC-BR', 4: 'BC-BCR', 5: 'BR-BCR'}
#pair_model_ids = dict([(v,k) for k,v in pair_model.items()])
rows = []
text_counter = Counter()
for infile in files:
    df = dfs[infile]
    #df['pairid'] = (np.arange(len(df))+1).tolist()
    columns = list(df.columns)
    annotators = []
    base = os.path.basename(infile)
    basename = os.path.splitext(base)[0]
    for i in [5, 9, 13]:
        if columns[i].startswith('Unnamed'):
            name = 'Unnamed_' + basename
        else:
            name = columns[i].title()
        annotators.append(name)

    keys = df[columns[-1]].values # unique id of pairwise samples
    ratings = df[[columns[i] for i in[3, 4, 7, 8, 11, 12]]].values
    texts_a = df[columns[0]].values
    texts_b = df[columns[1]].values
    relevance = df[[columns[i] for i in[2,6,10]]].values
    #pairs = df[columns[-1]].values
    
    #print(keys)
    
    for line_i, key in enumerate(keys):
        if type(key) == str:
            parts = key.split('_')
            model_a = models[int(parts[1]) % 4]
            model_b = models[int(parts[2]) % 4]
            text_a = texts_a[line_i]
            text_b = texts_b[line_i]
            text_counter.update([text_a])
            text_counter.update([text_b]) 
            #print("model_a:", model_a)
            #print("model_b:", model_b)
            #print()
            pairtxt=""
            lookup = str(model_a)+'-'+str(model_b)
            pairtxt = lookup
            
            if not np.isnan(ratings[line_i, 0]):
                rows.append([text_a, basename, model_a, annotators[0], ratings[line_i, 0], relevance[line_i,0], key, pairtxt])
            if not np.isnan(ratings[line_i, 2]):
                rows.append([text_a, basename, model_a, annotators[1], ratings[line_i, 2] , relevance[line_i,1], key,  pairtxt])
            if not np.isnan(ratings[line_i, 4]):                
                rows.append([text_a, basename, model_a, annotators[2], ratings[line_i, 4], relevance[line_i,2], key, pairtxt])
            if not np.isnan(ratings[line_i, 1]):
                rows.append([text_b, basename, model_b, annotators[0], ratings[line_i, 1], relevance[line_i,0], key,  pairtxt])
            if not np.isnan(ratings[line_i, 3]): 
                rows.append([text_b, basename, model_b, annotators[1], ratings[line_i, 3], relevance[line_i,1], key, pairtxt])
            if not np.isnan(ratings[line_i, 5]):    
                rows.append([text_b, basename, model_b, annotators[2], ratings[line_i, 5], relevance[line_i,2], key,  pairtxt])
            
df_out = pd.DataFrame(rows, columns=['text', 'domain', 'model', 'annotator', 'rating', 'relevance','key', 'pairtxt'])
df_out.head(20)

,text,domain,model,annotator,rating,relevance,key,pairtxt
0,The food truck scene at SXSW in Austin this we...,clickbait,BR,Gabe,3.0,neither,16_102_389,BR-BC
1,The food truck scene at SXSW in Austin this we...,clickbait,BR,Prabha,3.0,neither,16_102_389,BR-BC
2,The food truck scene at SXSW in Austin this we...,clickbait,BR,Unnamed_clickbait,4.0,B,16_102_389,BR-BC
3,The food and beverage industry is moving quick...,clickbait,BC,Gabe,3.0,neither,16_102_389,BR-BC
4,The food and beverage industry is moving quick...,clickbait,BC,Prabha,4.0,neither,16_102_389,BR-BC
5,The food and beverage industry is moving quick...,clickbait,BC,Unnamed_clickbait,5.0,B,16_102_389,BR-BC
6,"The potato is one nutritious staple, and it's ...",clickbait,BR,Gabe,3.0,A,37_698_56,BR-B
7,"The potato is one nutritious staple, and it's ...",clickbait,BR,Prabha,3.0,A,37_698_56,BR-B
8,"The potato is one nutritious staple, and it's ...",clickbait,BR,Unnamed_clickbait,4.0,A,37_698_56,BR-B
9,"The potato is also known as the ""pumpkin,"" and...",clickbait,B,Gabe,4.0,A,37_698_56,BR-B


In [13]:
len(df_out)

19486

In [14]:
set(df_out['pairtxt'].values)

{'B-BC',
 'B-BCR',
 'B-BR',
 'BC-B',
 'BC-BCR',
 'BC-BR',
 'BCR-B',
 'BCR-BC',
 'BCR-BR',
 'BR-B',
 'BR-BC',
 'BR-BCR'}

In [15]:
ratings_per_text = defaultdict(list)
for row in rows:
    text = row[0]
    rating = row[-4]
    ratings_per_text[text].append(rating)
print(len(ratings_per_text))

1361


In [16]:
ratings_per_text

defaultdict(list,
            {"The food truck scene at SXSW in Austin this weekend was a big thing.\n\nBut as I sat in the restaurant-style bar at The Bierhaus in San Antonio, I was thinking about how many of those folks didn't even know they were": [3.0,
              3.0,
              4.0,
              3.0,
              3.0,
              3.0,
              3.0,
              3.0,
              3.0],
             "The food and beverage industry is moving quickly and in the right direction.\n\nIt's a trend, and the trend is good.\n\nThe food and beverage sector is projected to grow at a CAGR of 9% to 7.7% in": [3.0,
              4.0,
              5.0,
              3.0,
              4.0,
              4.0,
              3.0,
              4.0,
              2.0],
             "The potato is one nutritious staple, and it's an incredibly healthy and nutritious food. But there are a few things you need to know about it and when to eat it.\n\nPotato Facts & Nutrition\n\nWhat is Pot

In [17]:
# convert everything to indices
text_counter = Counter(df_out['text'].values)
text_index = dict(zip(sorted(text_counter), range( len(text_counter))))
text_indices = ['I' + str(text_index[t]) for t in df_out['text'].values]
df_out['item_id'] = text_indices

model_counter = Counter(df_out['model'].values)
model_index = dict(zip(sorted(model_counter), range(len(model_counter))))
model_indices = [model_index[t] for t in df_out['model'].values]

annotator_counter = Counter(df_out['annotator'].values)
annotator_index = dict(zip(sorted(annotator_counter), range(len(annotator_counter))))
annotator_indices = ['W' + str(annotator_index[t]) for t in df_out['annotator'].values]
df_out['worker_id'] = annotator_indices

ratings = [float(r) for r in df_out['rating'].values]
print(len(text_indices), len(model_indices), len(annotator_indices), len(ratings))

rating_counter = Counter(ratings)
print(len(rating_counter))
df_out.head()

19486 19486 19486 19486
5


,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,worker_id
0,The food truck scene at SXSW in Austin this we...,clickbait,BR,Gabe,3.0,neither,16_102_389,BR-BC,I1063,W4
1,The food truck scene at SXSW in Austin this we...,clickbait,BR,Prabha,3.0,neither,16_102_389,BR-BC,I1063,W7
2,The food truck scene at SXSW in Austin this we...,clickbait,BR,Unnamed_clickbait,4.0,B,16_102_389,BR-BC,I1063,W12
3,The food and beverage industry is moving quick...,clickbait,BC,Gabe,3.0,neither,16_102_389,BR-BC,I1053,W4
4,The food and beverage industry is moving quick...,clickbait,BC,Prabha,4.0,neither,16_102_389,BR-BC,I1053,W7


In [18]:
len(df_out)

19486

In [19]:
print(set(df_out['model'].values))

{'BC', 'B', 'BR', 'BCR'}


In [20]:
print(set(df_out['rating']))

{1.0, 2.0, 3.0, 4.0, 5.0}


### Mapping Fluency rating score to category
x < 3 : Poor

x ==3 : Moderate

x > 3 : High

In [21]:
fluency_cat = []
for r in df_out.rating.values:
    cat = ""
    if int(r)==3:
        cat = "moderate"
    elif int(r) < 3:
        cat="poor"
    elif int(r) > 3:
        cat="high"
    fluency_cat.append(cat)
        
df_out['fluency_cat'] = np.array(fluency_cat)
df_out.head()

,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,worker_id,fluency_cat
0,The food truck scene at SXSW in Austin this we...,clickbait,BR,Gabe,3.0,neither,16_102_389,BR-BC,I1063,W4,moderate
1,The food truck scene at SXSW in Austin this we...,clickbait,BR,Prabha,3.0,neither,16_102_389,BR-BC,I1063,W7,moderate
2,The food truck scene at SXSW in Austin this we...,clickbait,BR,Unnamed_clickbait,4.0,B,16_102_389,BR-BC,I1063,W12,high
3,The food and beverage industry is moving quick...,clickbait,BC,Gabe,3.0,neither,16_102_389,BR-BC,I1053,W4,moderate
4,The food and beverage industry is moving quick...,clickbait,BC,Prabha,4.0,neither,16_102_389,BR-BC,I1053,W7,high


### Mapping relevance to its binary value
x == 'neither' --> No (0)

x == A & model == pairtxt[0] --> Yes (1); else --> No (0)

x == B & model == pairtxt[1] --> Yes (1) ; else --> No (0)


In [22]:
relev_bin = []
for i, r in enumerate(df_out.relevance.values):
    model_i = str(df_out.model.iloc[i])
    pair_i = df_out.pairtxt.iloc[i].split('-')
    
    rbin = 0
    if str(r)=='neither':
        rbin = 0
    elif str(r)=='both':
        rbin = 1
    elif str(r)=='A':
        if (str(r)=='A') and (pair_i[0] == model_i):
            rbin = 1
        else:
            rbin = 0
    elif str(r)=='B':
        if (str(r)=='B') and (pair_i[1] == model_i):
            #print(model_i)
            #print(pair_i[1])
            #print()
            rbin = 1
        else:
            rbin = 0
    relev_bin.append(rbin)
        
df_out['relevance_bin'] = np.array(relev_bin)
df_out.head()

,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,worker_id,fluency_cat,relevance_bin
0,The food truck scene at SXSW in Austin this we...,clickbait,BR,Gabe,3.0,neither,16_102_389,BR-BC,I1063,W4,moderate,0
1,The food truck scene at SXSW in Austin this we...,clickbait,BR,Prabha,3.0,neither,16_102_389,BR-BC,I1063,W7,moderate,0
2,The food truck scene at SXSW in Austin this we...,clickbait,BR,Unnamed_clickbait,4.0,B,16_102_389,BR-BC,I1063,W12,high,0
3,The food and beverage industry is moving quick...,clickbait,BC,Gabe,3.0,neither,16_102_389,BR-BC,I1053,W4,moderate,0
4,The food and beverage industry is moving quick...,clickbait,BC,Prabha,4.0,neither,16_102_389,BR-BC,I1053,W7,high,0


In [23]:
model_code_map = {'BR': 1, 'BC':2 , 'BCR': 3, 'B': 4}
model_codes = [model_code_map[m] for m in df_out['model'].values]
df_out['model_code'] = model_codes
df_out.head()

,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,worker_id,fluency_cat,relevance_bin,model_code
0,The food truck scene at SXSW in Austin this we...,clickbait,BR,Gabe,3.0,neither,16_102_389,BR-BC,I1063,W4,moderate,0,1
1,The food truck scene at SXSW in Austin this we...,clickbait,BR,Prabha,3.0,neither,16_102_389,BR-BC,I1063,W7,moderate,0,1
2,The food truck scene at SXSW in Austin this we...,clickbait,BR,Unnamed_clickbait,4.0,B,16_102_389,BR-BC,I1063,W12,high,0,1
3,The food and beverage industry is moving quick...,clickbait,BC,Gabe,3.0,neither,16_102_389,BR-BC,I1053,W4,moderate,0,2
4,The food and beverage industry is moving quick...,clickbait,BC,Prabha,4.0,neither,16_102_389,BR-BC,I1053,W7,high,0,2


In [24]:
text = df_out['text'].values
text_clean = [t.replace('<|endoftext|>', '') for t in text]
df_out['text_clean'] = np.array(text_clean)

In [25]:
text_clean_tok = [t.split() for t in text_clean]

In [26]:
lens=[]
for tok in text_clean_tok:
    lens.append(len(tok))
df_out['text_len'] = np.array(lens)

In [27]:
np.mean(lens)

60.84198912039413

In [28]:
np.std(lens)

10.986738865523018

In [29]:
prefix = [' '.join(t.split()[:2]) for t in text_clean]
df_out['prefix'] = np.array(prefix)

In [30]:
df_out.head()

,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,worker_id,fluency_cat,relevance_bin,model_code,text_clean,text_len,prefix
0,The food truck scene at SXSW in Austin this we...,clickbait,BR,Gabe,3.0,neither,16_102_389,BR-BC,I1063,W4,moderate,0,1,The food truck scene at SXSW in Austin this we...,42,The food
1,The food truck scene at SXSW in Austin this we...,clickbait,BR,Prabha,3.0,neither,16_102_389,BR-BC,I1063,W7,moderate,0,1,The food truck scene at SXSW in Austin this we...,42,The food
2,The food truck scene at SXSW in Austin this we...,clickbait,BR,Unnamed_clickbait,4.0,B,16_102_389,BR-BC,I1063,W12,high,0,1,The food truck scene at SXSW in Austin this we...,42,The food
3,The food and beverage industry is moving quick...,clickbait,BC,Gabe,3.0,neither,16_102_389,BR-BC,I1053,W4,moderate,0,2,The food and beverage industry is moving quick...,38,The food
4,The food and beverage industry is moving quick...,clickbait,BC,Prabha,4.0,neither,16_102_389,BR-BC,I1053,W7,high,0,2,The food and beverage industry is moving quick...,38,The food


In [33]:
df_out.columns

Index(['text', 'domain', 'model', 'annotator', 'rating', 'relevance', 'key',
       'pairtxt', 'item_id', 'worker_id', 'fluency_cat', 'relevance_bin',
       'model_code', 'text_clean', 'text_len', 'prefix'],
      dtype='object')

In [34]:
set(df_out['pairtxt'].values)

{'B-BC',
 'B-BCR',
 'B-BR',
 'BC-B',
 'BC-BCR',
 'BC-BR',
 'BCR-B',
 'BCR-BC',
 'BCR-BR',
 'BR-B',
 'BR-BC',
 'BR-BCR'}

In [32]:
outdir = os.path.join('../data', 'pplm-large')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_out.to_csv(os.path.join(outdir, 'uber_all_paired.csv'))

In [36]:
excluded = ['clickbait', 'positive', 'negative']
uber_topic_paired = df_out[~df_out['domain'].isin(excluded)]

In [37]:
set(uber_topic_paired['domain'].values)

{'computers', 'legal', 'military', 'politics', 'religion', 'science', 'space'}

In [38]:
outdir = os.path.join('../data', 'pplm-large')
if not os.path.exists(outdir):
    os.makedirs(outdir)

uber_topic_paired.to_csv(os.path.join(outdir, 'uber_topic_paired.csv'))

Because evaluating with CTRLEval and UniEval are slow, we remove redundancy here

In [46]:
base_url = "../data/pplm-large"
file = "uber_topic_paired_auto.csv"

df_raw = pd.read_csv(os.path.join(base_url, file), header=0, index_col=0)     

In [47]:
df_raw.head()

,Unnamed: 0.1,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,...,bert_precision,bert_recall,ctc_e_consisten,ctc_e_relev,ctc_d_consisten,ctc_d_relev,ctc_r_consisten,ctc_r_relev,cohere_met,consist_met
0,1620,"<|endoftext|>In summary, we have seen that the...",computers,BCR,Raymis,4,B,19_131_633,BCR-BC,I260,...,0.597291,0.673076,0.678896,0.444783,0.742355,0.076522,0.115709,0.073466,-2.501122,-3.567711
1,1621,"<|endoftext|>In summary, we have seen that the...",computers,BCR,Gabe,4,B,19_131_633,BCR-BC,I260,...,0.597291,0.673076,0.678896,0.444783,0.742355,0.076522,0.115709,0.073466,-2.501122,-3.567711
2,1622,"<|endoftext|>In summary, we have seen that the...",computers,BCR,Ben,4,B,19_131_633,BCR-BC,I260,...,0.597291,0.673076,0.678896,0.444783,0.742355,0.076522,0.115709,0.073466,-2.501122,-3.567711
3,1623,"<|endoftext|>In summary, you should be able to...",computers,BC,Raymis,3,B,19_131_633,BCR-BC,I263,...,0.606844,0.673815,0.678716,0.459447,0.705316,0.072039,0.113510,0.039607,-2.294639,-3.122753
4,1624,"<|endoftext|>In summary, you should be able to...",computers,BC,Gabe,2,B,19_131_633,BCR-BC,I263,...,0.606844,0.673815,0.678716,0.459447,0.705316,0.072039,0.113510,0.039607,-2.294639,-3.122753


In [49]:
len(df_raw)

14626

In [48]:
len(set(df_raw['text_clean'].values))

890

In [59]:
df_nored = df_raw.drop_duplicates(subset=['domain', 'text_clean', 'prefix'], keep='first')

In [60]:
len(df_nored)

1361

In [61]:
df_nored.head()

,Unnamed: 0.1,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,...,bert_precision,bert_recall,ctc_e_consisten,ctc_e_relev,ctc_d_consisten,ctc_d_relev,ctc_r_consisten,ctc_r_relev,cohere_met,consist_met
0,1620,"<|endoftext|>In summary, we have seen that the...",computers,BCR,Raymis,4,B,19_131_633,BCR-BC,I260,...,0.597291,0.673076,0.678896,0.444783,0.742355,0.076522,0.115709,0.073466,-2.501122,-3.567711
3,1623,"<|endoftext|>In summary, you should be able to...",computers,BC,Raymis,3,B,19_131_633,BCR-BC,I263,...,0.606844,0.673815,0.678716,0.459447,0.705316,0.072039,0.113510,0.039607,-2.294639,-3.122753
6,1626,"<|endoftext|>In summary, the study results con...",computers,BC,Raymis,3,neither,18_389_15,BC-BCR,I242,...,0.602687,0.674949,0.656664,0.431520,0.755405,0.074090,0.298668,0.154131,-2.860359,-2.771099
9,1629,"<|endoftext|>In summary, we conclude the repor...",computers,BCR,Raymis,2,neither,18_389_15,BC-BCR,I250,...,0.588530,0.670989,0.702641,0.465752,0.800716,0.079620,0.084757,0.055465,-2.865513,-3.013949
12,1632,<|endoftext|>This essay discusses a new theory...,computers,BCR,Raymis,4,A,6_271_178,BCR-BR,I679,...,0.598946,0.676975,0.675183,0.449703,0.783897,0.078932,-0.221857,-0.138330,-2.675349,-3.426051


In [62]:
outdir = os.path.join('../data', 'pplm-large')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df_nored.to_csv(os.path.join(outdir, 'uber_topic_paired_nored_raw.csv'))

Read UBER data no redundancy that includes CTRLEval Relevance scores and UniEval scores (not yet added into the pair dataset)

In [63]:
base_url = "../data/pplm-large"
file1 = "uber_topic_paired_auto.csv"
file2 = "uber_topic_paired_nored.csv"

df1 = pd.read_csv(os.path.join(base_url, file1), header=0, index_col=0) 
df2 = pd.read_csv(os.path.join(base_url, file2), header=0, index_col=0) 

In [64]:
len(df1)

14626

In [65]:
len(df2)

1361

In [66]:
df2.head(3)

,text,domain,model,annotator,rating,relevance,key,pairtxt,item_id,worker_id,...,ctc_r_consisten,ctc_r_relev,cohere_met,consist_met,relev_met,coherence_unieval,consistency_unieval,fluency_unieval,relevance_unieval,overall_unieval
0,"<|endoftext|>In summary, we have seen that the...",computers,BCR,Raymis,4,B,19_131_633,BCR-BC,I260,W8,...,0.115709,0.073466,-2.501122,-3.567711,0.005582,0.497245,0.710580,0.779340,0.535837,0.630751
3,"<|endoftext|>In summary, you should be able to...",computers,BC,Raymis,3,B,19_131_633,BCR-BC,I263,W8,...,0.113510,0.039607,-2.294639,-3.122753,0.116588,0.680262,0.734453,0.656155,0.595454,0.666581
6,"<|endoftext|>In summary, the study results con...",computers,BC,Raymis,3,neither,18_389_15,BC-BCR,I242,W8,...,0.298668,0.154131,-2.860359,-2.771099,0.000957,0.623455,0.696262,0.865100,0.556272,0.685272


In [90]:
domain2 = df2['domain'].values
domain2

array(['computers', 'computers', 'computers', ..., 'space', 'space',
       'space'], dtype=object)

In [92]:
len(domain2)

1361

In [91]:
prefix2 = df2['prefix'].values
len(prefix2)

1361

In [93]:
prefix2[:5]

array(['In summary,', 'In summary,', 'In summary,', 'In summary,',
       'This essay'], dtype=object)

In [94]:
text2 = df2['text_clean'].values
len(text2)

1361

In [95]:
relev_sc = []
coh_unieval = []
cons_unieval = []
flu_unieval = []
rel_unieval = []
overal_unieval = []
for i, (d, p, t) in enumerate(zip(domain2, prefix2, text2)):
    for j, (d1, p1, t1) in enumerate(zip(df1['domain'].values, df1['prefix'].values, df1['text_clean'].values)):
        if (d1==d) and (p1==p) and (t1==t):
            relev_sc.append(df2.iloc[i].relev_met)
            coh_unieval.append(df2.iloc[i].coherence_unieval)
            cons_unieval.append(df2.iloc[i].consistency_unieval)
            flu_unieval.append(df2.iloc[i].fluency_unieval)
            rel_unieval.append(df2.iloc[i].relevance_unieval)
            overal_unieval.append(df2.iloc[i].overall_unieval)

In [96]:
relev_sc = np.array(relev_sc)
coh_unieval = np.array(coh_unieval)
cons_unieval = np.array(cons_unieval)
flu_unieval = np.array(flu_unieval)
rel_unieval = np.array(rel_unieval)
overal_unieval = np.array(overal_unieval)

In [98]:
len(coh_unieval)

14626

In [99]:
df1['relev_met'] = relev_sc
df1['coherence_unieval'] = coh_unieval
df1['consistency_unieval'] = cons_unieval
df1['fluency_unieval'] = flu_unieval
df1['relevance_unieval'] = rel_unieval
df1['overall_unieval'] = overal_unieval

In [100]:
outdir = os.path.join('../data', 'pplm-large')
if not os.path.exists(outdir):
    os.makedirs(outdir)

df1.to_csv(os.path.join(outdir, 'uber_topic_paired_auto_.csv'))